In [1]:
import pandas as pd

# Load the saved CSV file
data = pd.read_csv('processed_data.csv')


In [2]:
data

,X_Start,Y_Start,X_End,Y_end,Z,meanLineIntensity,varLineIntensity,meanNumPixels15,varNumPixels15,meanNumPixels30,...,varNumPixels60,Median_meltLength,varMeltLength,Overhang,Wall,Power,d_edge,d_over,d_col,d_over_z
0,0,0.05,1.5,0.05,0.50,1.828807e+06,5.902107e+09,4884.797753,9.249608e+05,1195.752809,...,18681.207099,42,47.422625,0,0,180,0.05,3.3,0.0,1.65
1,0,0.10,1.5,0.10,0.50,1.914433e+06,5.807522e+09,6706.269663,1.771474e+06,1619.438202,...,26267.830695,41,52.967314,0,0,180,0.05,3.3,0.0,1.65
2,0,0.15,1.5,0.15,0.50,1.910230e+06,4.765438e+09,6109.460674,7.615990e+05,1612.179775,...,39714.737487,43,77.342952,0,0,180,0.10,3.3,0.0,1.65
3,0,0.20,1.5,0.20,0.50,1.905874e+06,7.406520e+09,5864.943820,1.277515e+06,1534.775281,...,22738.784729,43,42.477017,0,0,180,0.15,3.3,0.0,1.65
4,0,0.25,1.5,0.25,0.50,1.844209e+06,3.619133e+09,5074.494382,3.498485e+05,1490.022472,...,25185.487487,41,25.244893,0,0,180,0.20,3.3,0.0,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,0,3.15,1.5,3.15,0.45,1.859401e+06,7.614869e+09,5572.820225,2.422673e+06,1103.820225,...,7935.262002,37,40.245403,0,0,180,0.20,3.3,0.0,1.65
1808,0,3.20,1.5,3.20,0.45,1.933627e+06,6.933997e+09,6258.314607,1.999094e+06,1383.382022,...,20979.463228,40,133.660368,0,0,180,0.15,3.3,0.0,1.65
1809,0,3.25,1.5,3.25,0.45,1.880439e+06,7.908295e+09,5551.235955,2.410604e+06,1299.382022,...,18303.926200,40,60.948927,0,0,180,0.10,3.3,0.0,1.65
1810,0,3.30,1.5,3.30,0.45,1.889719e+06,1.324928e+10,5593.842697,2.631757e+06,1325.471910,...,15455.863126,41,60.363126,0,0,180,0.05,3.3,0.0,1.65


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader

# Define target and input variables
target_cols = ['Median_meltLength', 'meanNumPixels15']
input_cols = ['Power', 'Z', 'Y_Start', 'Wall', 'd_edge', 'd_over', 'd_col', 'd_over_z']

# Extract features and targets
X = data[input_cols].values
y = data[target_cols].values

# Create a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Function to prepare data loaders for a specific fold
def prepare_fold_data(X, y, train_idx, val_idx, batch_size=32):
    # Scale the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X[train_idx])
    X_val = scaler.transform(X[val_idx])
    
    # Create datasets
    train_dataset = CustomDataset(X_train, y[train_idx])
    val_dataset = CustomDataset(X_val, y[val_idx])
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False
    )
    
    return train_loader, val_loader, scaler

# Set up K-Fold cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store fold indices for later use
fold_indices = []
for train_idx, val_idx in kf.split(X):
    fold_indices.append((train_idx, val_idx))


In [4]:
import torch
import torch.nn as nn
from itertools import combinations_with_replacement, combinations

class PolynomialRegressor(nn.Module):
    def __init__(self, input_dim=8, output_dim=2, degree=3):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.degree = degree
        
        # Calculate number of features after polynomial expansion
        # Original features + squares + double interactions + triple interactions
        n_original = input_dim
        n_squares = input_dim  # x1^2, x2^2, etc.
        n_double_interactions = len(list(combinations_with_replacement(range(input_dim), 2))) - input_dim  # x1*x2, x1*x3, etc.
        n_triple_interactions = len(list(combinations(range(input_dim), 3)))  # x1*x2*x3, etc.
        
        self.expanded_dim = n_original + n_squares + n_double_interactions + n_triple_interactions
        
        print(f"Feature space dimensionality:")
        print(f"Original features: {n_original}")
        print(f"Squared terms: {n_squares}")
        print(f"Double interactions: {n_double_interactions}")
        print(f"Triple interactions: {n_triple_interactions}")
        print(f"Total features: {self.expanded_dim}")
        
        # Linear layer for the expanded features
        self.linear = nn.Linear(self.expanded_dim, output_dim)
        
    def compute_polynomial_features(self, x):
        batch_size = x.size(0)
        
        # Original features
        features = [x]
        
        # Squared terms
        squares = x ** 2
        features.append(squares)
        
        # Double interaction terms
        double_interactions = []
        for i in range(self.input_dim):
            for j in range(i+1, self.input_dim):
                interaction = x[:, i] * x[:, j]
                double_interactions.append(interaction.unsqueeze(1))
        
        if double_interactions:
            double_interactions_tensor = torch.cat(double_interactions, dim=1)
            features.append(double_interactions_tensor)
        
        # Triple interaction terms
        triple_interactions = []
        for i, j, k in combinations(range(self.input_dim), 3):
            interaction = x[:, i] * x[:, j] * x[:, k]
            triple_interactions.append(interaction.unsqueeze(1))
        
        if triple_interactions:
            triple_interactions_tensor = torch.cat(triple_interactions, dim=1)
            features.append(triple_interactions_tensor)
        
        # Concatenate all features
        return torch.cat(features, dim=1)
    
    def get_feature_names(self, input_names=None):
        """
        Returns the names of all features after polynomial expansion
        """
        if input_names is None:
            input_names = [f'x{i}' for i in range(self.input_dim)]
            
        feature_names = []
        
        # Original features
        feature_names.extend(input_names)
        
        # Squared terms
        feature_names.extend([f'{x}²' for x in input_names])
        
        # Double interaction terms
        for i in range(self.input_dim):
            for j in range(i+1, self.input_dim):
                feature_names.append(f'{input_names[i]}*{input_names[j]}')
        
        # Triple interaction terms
        for i, j, k in combinations(range(self.input_dim), 3):
            feature_names.append(f'{input_names[i]}*{input_names[j]}*{input_names[k]}')
        
        return feature_names
    
    def forward(self, x):
        # Compute polynomial features
        x_poly = self.compute_polynomial_features(x)
        # Apply linear regression on expanded features
        return self.linear(x_poly)
    
    def get_feature_importance(self, input_names=None):
        """
        Returns the importance (weight magnitude) of each feature
        """
        if input_names is None:
            input_names = [f'x{i}' for i in range(self.input_dim)]

        weights = self.linear.weight.detach().cpu()  # Shape: [output_dim, expanded_dim]
        importance = torch.abs(weights).mean(dim=0)  # Average across output dimensions

        feature_names = self.get_feature_names(input_names)  # Use the provided input names
        importance_dict = {name: float(imp) for name, imp in zip(feature_names, importance)}

        # Sort by importance
        importance_dict = dict(sorted(importance_dict.items(), key=lambda x: x[1], reverse=True))

        return importance_dict


In [5]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd

def plot_feature_importance(importance_across_folds, save_dir='./plots'):
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    
    # Calculate mean and std of importance across folds
    mean_importance = {}
    std_importance = {}
    
    all_features = importance_across_folds[0].keys()
    
    for feature in all_features:
        values = [fold_imp[feature] for fold_imp in importance_across_folds]
        mean_importance[feature] = np.mean(values)
        std_importance[feature] = np.std(values)
    
    # Sort by mean importance
    sorted_features = sorted(mean_importance.keys(), 
                           key=lambda x: mean_importance[x], 
                           reverse=True)
    
    # Create plot
    plt.figure(figsize=(15, 10))
    y_pos = np.arange(len(sorted_features))
    
    # Plot bars with error bars
    plt.barh(y_pos, 
            [mean_importance[feat] for feat in sorted_features],
            xerr=[std_importance[feat] for feat in sorted_features],
            capsize=5)
    
    plt.yticks(y_pos, sorted_features)
    plt.xlabel('Feature Importance (Mean Absolute Weight)')
    plt.title('Feature Importance Across Folds')
    
    # Add grid for better readability
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    # Save plot
    plt.savefig(f'{save_dir}/feature_importance.png')
    plt.close()

def plot_correlation_matrix(feature_matrix, feature_names, save_dir='./plots'):
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    
    # Calculate correlation matrix
    corr_matrix = np.corrcoef(feature_matrix.T)
    
    # Create plot
    plt.figure(figsize=(15, 12))
    sns.heatmap(corr_matrix, 
                xticklabels=feature_names, 
                yticklabels=feature_names,
                cmap='RdBu_r', 
                center=0,
                annot=True, 
                fmt='.2f', 
                square=True)
    
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.savefig(f'{save_dir}/feature_correlation.png')
    plt.close()

def run_cross_validation(model, fold_indices, X, y, num_epochs=100):
    all_metrics = []
    importance_across_folds = []
    feature_names = model.get_feature_names(['Power', 'Z', 'Y_Start', 'Wall', 'd_edge', 'd_over', 'd_col', 'd_over_z'])
    
    print("\nInitial feature space analysis:")
    print(f"Number of features after expansion: {model.expanded_dim}")
    print("\nFeature types:")
    for name in feature_names:
        print(f"- {name}")
    
    for fold, (train_idx, val_idx) in enumerate(fold_indices):
        print(f'\nTraining fold {fold+1}')
        
        # Prepare data for this fold
        train_loader, val_loader, scaler = prepare_fold_data(X, y, train_idx, val_idx, batch_size=len(train_idx))
        
        # Initialize new model for this fold
        fold_model = PolynomialRegressor(input_dim=8, output_dim=2)
        criterion = nn.MSELoss()
        
        # Initialize L-BFGS optimizer
        optimizer = torch.optim.LBFGS(fold_model.parameters(), 
                                    lr=1.0,
                                    max_iter=20,
                                    history_size=10,
                                    line_search_fn='strong_wolfe')
        
        # Train the model
        fold_model, history = train_model(
            model=fold_model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            num_epochs=num_epochs,
            fold=fold
        )
        
        # Plot training history
        plot_training_history(history, fold)
        
        # Calculate and store metrics
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        metrics = calculate_metrics(fold_model, val_loader, device)
        all_metrics.append(metrics)
        
        feature_names = ['Power', 'Z', 'Y_Start', 'Wall', 'd_edge', 'd_over', 'd_col', 'd_over_z']

        
        # Get feature importance for this fold
        fold_importance = fold_model.get_feature_importance(input_names=feature_names)
        importance_across_folds.append(fold_importance)
        
        # Print metrics for this fold
        print(f'\nMetrics for Fold {fold+1}:')
        for target, target_metrics in metrics.items():
            print(f'\n{target}:')
            print(f'R² Score: {target_metrics["r2"]:.4f}')
            print(f'RMSE: {target_metrics["rmse"]:.4f}')
            print(f'MAE: {target_metrics["mae"]:.4f}')
        
        # Print top 5 most important features for this fold
        print(f'\nTop 5 most important features for fold {fold+1}:')
        for idx, (feature, importance) in enumerate(list(fold_importance.items())[:5]):
            print(f"{idx+1}. {feature}: {importance:.4f}")
        
        # Generate feature matrix for correlation analysis
        if fold == 0:  # Do this only for first fold to avoid redundancy
            # Get polynomial features for the training data
            X_train = X[train_idx]
            X_train_tensor = torch.FloatTensor(X_train)
            poly_features = fold_model.compute_polynomial_features(X_train_tensor).detach().numpy()
            
            # Plot correlation matrix
            plot_correlation_matrix(poly_features, feature_names)
    
    # Plot overall feature importance
    plot_feature_importance(importance_across_folds)
    
    # Print average metrics across all folds
    print('\nAverage metrics across all folds:')
    for target in ['Median_meltLength', 'meanNumPixels15']:
        avg_r2 = np.mean([metrics[target]['r2'] for metrics in all_metrics])
        avg_rmse = np.mean([metrics[target]['rmse'] for metrics in all_metrics])
        avg_mae = np.mean([metrics[target]['mae'] for metrics in all_metrics])
        
        print(f'\n{target}:')
        print(f'Average R² Score: {avg_r2:.4f} ± {np.std([metrics[target]["r2"] for metrics in all_metrics]):.4f}')
        print(f'Average RMSE: {avg_rmse:.4f} ± {np.std([metrics[target]["rmse"] for metrics in all_metrics]):.4f}')
        print(f'Average MAE: {avg_mae:.4f} ± {np.std([metrics[target]["mae"] for metrics in all_metrics]):.4f}')
    
    # Print average feature importance across folds
    print('\nAverage feature importance across folds:')
    avg_importance = {}
    for feature in importance_across_folds[0].keys():
        values = [fold_imp[feature] for fold_imp in importance_across_folds]
        avg_importance[feature] = np.mean(values)
    
    # Sort and print top 10 features
    sorted_features = sorted(avg_importance.items(), key=lambda x: x[1], reverse=True)
    print('\nTop 10 most important features (averaged across folds):')
    for feature, importance in sorted_features[:10]:
        values = [fold_imp[feature] for fold_imp in importance_across_folds]
        print(f"{feature}: {np.mean(values):.4f} ± {np.std(values):.4f}")
    
    return all_metrics, importance_across_folds



In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pathlib import Path

def closure(model, X_batch, y_batch, criterion, optimizer):
    optimizer.zero_grad()
    output = model(X_batch)
    loss = criterion(output, y_batch)
    loss.backward()
    return loss

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, 
                early_stopping_patience=1000, fold=0, save_dir='./models'):
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    best_val_loss = float('inf')
    patience_counter = 0
    training_history = {'train_loss': [], 'val_loss': []}
    best_model_path = f'{save_dir}/best_model_fold_{fold+1}.pt'
    
    # Convert training data to single batch for L-BFGS
    X_train_all = []
    y_train_all = []
    for X_batch, y_batch in train_loader:
        X_train_all.append(X_batch)
        y_train_all.append(y_batch)
    X_train_all = torch.cat(X_train_all).to(device)
    y_train_all = torch.cat(y_train_all).to(device)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        
        # Define closure for L-BFGS
        def closure():
            optimizer.zero_grad()
            outputs = model(X_train_all)
            loss = criterion(outputs, y_train_all)
            loss.backward()
            return loss
        
        # Perform optimization step
        loss = optimizer.step(closure)
        train_loss = loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                val_loss += criterion(outputs, y_batch).item()
        
        # Calculate average losses
        avg_train_loss = train_loss  # For L-BFGS, this is already the average
        avg_val_loss = val_loss / len(val_loader)
        
        # Store losses
        training_history['train_loss'].append(avg_train_loss)
        training_history['val_loss'].append(avg_val_loss)
        
        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Save best model
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': best_val_loss,
            }, best_model_path)
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f'Early stopping triggered at epoch {epoch+1}')
                break
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], '
                  f'Train Loss: {avg_train_loss:.4f}, '
                  f'Val Loss: {avg_val_loss:.4f}')
    
    # Load best model before returning
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return model, training_history

In [7]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import os
from pathlib import Path

def calculate_metrics(model, data_loader, device):
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            y_true.append(y_batch.numpy())
            y_pred.append(outputs.cpu().numpy())
    
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    
    # Calculate metrics for each target variable
    metrics = {}
    for i, target_name in enumerate(['Median_meltLength', 'meanNumPixels15']):
        metrics[target_name] = {
            'r2': r2_score(y_true[:, i], y_pred[:, i]),
            'rmse': np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i])),
            'mae': mean_absolute_error(y_true[:, i], y_pred[:, i])
        }
    
    return metrics

def plot_training_history(history, fold, save_dir='./plots'):
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    
    plt.figure(figsize=(10, 6))
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Loss Evolution - Fold {fold+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{save_dir}/loss_evolution_fold_{fold+1}.png')
    plt.close()





In [8]:
feature_names = ['Power', 'Z', 'Y_Start', 'Wall', 'd_edge', 'd_over', 'd_col', 'd_over_z']

model = PolynomialRegressor(input_dim=8, output_dim=2)

metrics, importance = run_cross_validation(
    model=model,
    fold_indices=fold_indices,
    X=X,
    y=y,
    num_epochs=1000
)


Feature space dimensionality:
Original features: 8
Squared terms: 8
Double interactions: 28
Triple interactions: 56
Total features: 100

Initial feature space analysis:
Number of features after expansion: 100

Feature types:
- Power
- Z
- Y_Start
- Wall
- d_edge
- d_over
- d_col
- d_over_z
- Power²
- Z²
- Y_Start²
- Wall²
- d_edge²
- d_over²
- d_col²
- d_over_z²
- Power*Z
- Power*Y_Start
- Power*Wall
- Power*d_edge
- Power*d_over
- Power*d_col
- Power*d_over_z
- Z*Y_Start
- Z*Wall
- Z*d_edge
- Z*d_over
- Z*d_col
- Z*d_over_z
- Y_Start*Wall
- Y_Start*d_edge
- Y_Start*d_over
- Y_Start*d_col
- Y_Start*d_over_z
- Wall*d_edge
- Wall*d_over
- Wall*d_col
- Wall*d_over_z
- d_edge*d_over
- d_edge*d_col
- d_edge*d_over_z
- d_over*d_col
- d_over*d_over_z
- d_col*d_over_z
- Power*Z*Y_Start
- Power*Z*Wall
- Power*Z*d_edge
- Power*Z*d_over
- Power*Z*d_col
- Power*Z*d_over_z
- Power*Y_Start*Wall
- Power*Y_Start*d_edge
- Power*Y_Start*d_over
- Power*Y_Start*d_col
- Power*Y_Start*d_over_z
- Power*Wall*

Epoch [960/1000], Train Loss: 235165.9219, Val Loss: 248091.0469
Epoch [970/1000], Train Loss: 235165.9219, Val Loss: 248091.0469
Epoch [980/1000], Train Loss: 235165.9219, Val Loss: 248091.0469
Epoch [990/1000], Train Loss: 235165.9219, Val Loss: 248091.0469
Epoch [1000/1000], Train Loss: 235165.9219, Val Loss: 248091.0469

Metrics for Fold 1:

Median_meltLength:
R² Score: -2.3744
RMSE: 8.0992
MAE: 4.5114

meanNumPixels15:
R² Score: 0.4049
RMSE: 696.5692
MAE: 551.0764

Top 5 most important features for fold 1:
1. d_over_z: 889.0762
2. Z*d_col*d_over_z: 713.3776
3. d_col: 656.7336
4. Z*d_over*d_col: 513.1658
5. d_col*d_over_z: 379.4509


/home/juls/Adaptive-Filter-for-Acoustic-Signal-Segmentation/myenv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/juls/Adaptive-Filter-for-Acoustic-Signal-Segmentation/myenv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]



Training fold 2
Feature space dimensionality:
Original features: 8
Squared terms: 8
Double interactions: 28
Triple interactions: 56
Total features: 100
Epoch [10/1000], Train Loss: 236972.1562, Val Loss: 254101.2031
Epoch [20/1000], Train Loss: 233090.3594, Val Loss: 258402.4688
Epoch [30/1000], Train Loss: 232777.3125, Val Loss: 258373.2031
Epoch [40/1000], Train Loss: 232693.0625, Val Loss: 256851.1406
Epoch [50/1000], Train Loss: 232578.2031, Val Loss: 256955.5781
Epoch [60/1000], Train Loss: 232554.1250, Val Loss: 257122.8750
Epoch [70/1000], Train Loss: 232553.5156, Val Loss: 257072.2031
Epoch [80/1000], Train Loss: 232553.5156, Val Loss: 257072.2031
Epoch [90/1000], Train Loss: 232553.5156, Val Loss: 257072.2031
Epoch [100/1000], Train Loss: 232553.5156, Val Loss: 257072.2031
Epoch [110/1000], Train Loss: 232553.5156, Val Loss: 257072.2031
Epoch [120/1000], Train Loss: 232553.5156, Val Loss: 257072.2031
Epoch [130/1000], Train Loss: 232553.5156, Val Loss: 257072.2031
Epoch [140/

Epoch [180/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [190/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [200/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [210/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [220/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [230/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [240/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [250/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [260/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [270/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [280/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [290/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [300/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [310/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [320/1000], Train Loss: 236308.3750, Val Loss: 241039.5625
Epoch [330/1000], Train L

Epoch [370/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [380/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [390/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [400/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [410/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [420/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [430/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [440/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [450/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [460/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [470/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [480/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [490/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [500/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [510/1000], Train Loss: 230006.6094, Val Loss: 269306.5625
Epoch [520/1000], Train L

Epoch [570/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [580/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [590/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [600/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [610/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [620/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [630/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [640/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [650/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [660/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [670/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [680/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [690/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [700/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [710/1000], Train Loss: 233628.4688, Val Loss: 257300.7656
Epoch [720/1000], Train L